# Load data

In [1]:
! pip install pennylane
! pip install torch
! pip install torch_geometric
! pip install tensorflow
! pip install torchsummary


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 102.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import pennylane as qml
from pennylane import numpy as pnp
import jax
from jax import numpy as jnp
import optax
warnings.filterwarnings("ignore")

/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


In [3]:
df_train = pd.read_csv('/kaggle/input/housing/train.csv')
df_val = pd.read_csv('/kaggle/input/housing/val.csv')
df_test = pd.read_csv('/kaggle/input/housing/test.csv')

In [4]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [5]:
features_numerical = df_train.select_dtypes(exclude=['object', 'bool']).copy()
numerical_cols = features_numerical.columns.tolist()

numerical_cols.remove('price')

features_categorical = df_train.select_dtypes(include=['object', 'bool']).copy()
categorical_cols = features_categorical.columns.tolist()

categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
numerical_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols),
        ('num', numerical_transformer, numerical_cols)
    ],
    remainder='passthrough'
)

y_train = df_train['price'].copy()
y_val = df_val['price'].copy()

df_train = df_train.drop(['price'], axis = 1)
df_val = df_val.drop(['price'], axis = 1)

y_train.to_numpy()
y_val.to_numpy()

X_train = preprocessor.fit_transform(df_train)
X_val = preprocessor.transform(df_val)
X_test = preprocessor.transform(df_test)


# Quantum

In [79]:
import matplotlib.pyplot as plt
import pennylane as qml
from pennylane import numpy as pnp
import jax
from jax import numpy as jnp
import optax
import torch
import torch.nn as nn
import pennylane as qml
import random


In [80]:
import os
#from Uwb_dataset import import_from_files
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D, LSTM, Bidirectional
import tensorflow as tf
# Loads and Processes the data that will be used in QCNN and Hierarchical Classifier Training
import numpy as np
#import tensorflow as tf
from sklearn.decomposition import PCA
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
import matplotlib.pyplot as plt
import pennylane as qml
from pennylane import numpy as np
import autograd.numpy as anp
from pennylane.templates.embeddings import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.state_preparations import MottonenStatePreparation
from pennylane.templates import RandomLayers
from sklearn.preprocessing import normalize
import torch
device = ("cuda" if torch.cuda.is_available() else "cpu")

In [81]:
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_val_scaled = scaler_y.transform(y_val.values.reshape(-1, 1))

X_train_torch = torch.tensor(X_train, dtype = torch.float32).to(device)
X_val_torch = torch.tensor(X_val, dtype = torch.float32).to(device)
X_test_torch = torch.tensor(X_test, dtype = torch.float32).to(device)
y_train_torch = torch.tensor(y_train_scaled, dtype = torch.float32).to(device)
y_val_torch = torch.tensor(y_val_scaled, dtype = torch.float32).to(device)

In [82]:
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(69)

In [10]:
# import pennylane as qml

# def U_4(params, wires): # 3 params 2 qubit
#     qml.RZ(-np.pi / 2, wires = wires[1])
#     qml.CNOT(wires = [wires[1], wires[0]]) # (source, target)
#     qml.RZ(params[0], wires = wires[0])
#     qml.RY(params[1], wires = wires[1])
#     qml.CNOT(wires = [wires[0], wires[1]])
#     qml.RZ(params[2], wires = wires[1])
#     qml.CNOT(wires = [wires[1], wires[0]])
#     qml.RZ(np.pi / 2, wires = wires[0])
    
# def conv_layer1(U, params, Uname): 
#     if Uname == 'U_4': # parameter 3
#         U(params[0:3], wires=[0, 1])
#         U(params[3:6], wires=[2, 3])
#         U(params[6:9], wires=[4, 5])
#         U(params[9:12], wires=[6, 7])
#         U(params[12:15], wires=[8, 9])
        
#         U(params[15:18], wires=[1, 2])
#         U(params[18:21], wires=[3, 4])
#         U(params[21:24], wires=[5, 6])
#         U(params[24:27], wires=[7, 8])
#         U(params[27:30], wires=[9, 0])
        
# def U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires): # 15 params, Convolutional Circuit 10
#     qml.U3(*weights_0, wires=wires[0])
#     qml.U3(*weights_1, wires=wires[1])
#     qml.CNOT(wires=[wires[0], wires[1]])
#     qml.RY(weights_2, wires=wires[0])
#     qml.RZ(weights_3, wires=wires[1])
#     qml.CNOT(wires=[wires[1], wires[0]])
#     qml.RY(weights_4, wires=wires[0])
#     qml.CNOT(wires=[wires[0], wires[1]])
#     qml.U3(*weights_5, wires=wires[0])
#     qml.U3(*weights_6, wires=wires[1])
    
# # Unitary Ansatz for Pooling Layer
# def Pooling_ansatz1(weights_0, weights_1, wires): #2 params
#     qml.CRZ(weights_0, wires=[wires[0], wires[1]])
#     qml.PauliX(wires=wires[0])
#     qml.CRX(weights_1, wires=[wires[0], wires[1]])
    
    
# n_qubits = 10
# dev = qml.device("default.qubit", wires=n_qubits)
# @qml.qnode(dev, interface = "torch")
# def qnode(inputs, weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, weights_7, weights_8): # , weights_9, weights_10, weights_11, weights_12, weights_13, weights_14, weights_15, weights_16, weights_17
#     qml.AngleEmbedding(inputs, wires=range(n_qubits))
#     #qml.AmplitudeEmbedding(inputs, wires=range(n_qubits), normalize=True)
#     #qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
   
#     # QCNN
#     #--------------------------------------------------------- Convolutional Layer1 ---------------------------------------------------------#    
#     U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[0, 1])
#     U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[2, 3])
#     U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[4, 5])
#     U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[6, 7])
#     U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[8, 9])
    
#     U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[1, 2])
#     U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[3, 4])
#     U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[5, 6])
#     # U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[7, 0])
#     U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[7, 8])
#     U_SU4(weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, wires=[9, 0])

#     #--------------------------------------------------------- Pooling Layer1 ---------------------------------------------------------#
#     ## Pooling Circuit  Block 2 weights_7, weights_8
#     Pooling_ansatz1(weights_7, weights_8, wires=[0,1])
#     Pooling_ansatz1(weights_7, weights_8, wires=[2,3])
#     Pooling_ansatz1(weights_7, weights_8, wires=[4,5])
#     Pooling_ansatz1(weights_7, weights_8, wires=[6,7])
#     Pooling_ansatz1(weights_7, weights_8, wires=[8,9])

#     result = [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]
#     return result

In [11]:
# weight_shapes = {
#     "weights_0": 3,
#     "weights_1": 3,
#     "weights_2": 1,
#     "weights_3": 1,
#     "weights_4": 1,
#     "weights_5": 3,
#     "weights_6": 3,
#     "weights_7": 1,
#     "weights_8": 1,    
# }

# qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)

In [83]:
n_qubits = 6
dev = qml.device("default.qubit", wires = n_qubits)

@qml.qnode(dev, interface = "torch")
def quantum_circuit(inputs, weights):
    qml.AngleEmbedding(inputs, wires = range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires = range(n_qubits))
    # qml.StronglyEntanglingLayers(weights, wires = range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

weight_shapes = {"weights": (3, n_qubits)}

qlayer = qml.qnn.TorchLayer(quantum_circuit, weight_shapes)


In [92]:
class HybridQNN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            qlayer, # Sử dụng qlayer đã tạo
            nn.Linear(hidden_dim, out_dim)
        )
        # self.layers = []
        # in_features = in_dim
        # for hdim in hidden_dim:
        #     self.layers.append(nn.Linear(in_features, hdim))
        #     self.layers.append(nn.ReLU())
        #     self.layers.append(nn.Dropout(0.3))
        #     in_features = hdim
        # self.layers.append(nn.Linear(in_features, out_dim))
        # self.model = nn.Sequential(*self.layers)
    def forward(self, x):
        # x = self.classical1(x)
        # # x = self.quantum(x)
        # x = self.classical2(x)
        # return x

        return self.model(x)

model = HybridQNN(in_dim = X_train_torch.shape[1], hidden_dim = n_qubits, out_dim = 1).to(device)

In [93]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

In [94]:
model.apply(init_weights)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=1)


In [95]:
from torchsummary import summary as summary

summary(model, (27,)) # (model, input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 6]             168
        TorchLayer-2                    [-1, 6]               0
            Linear-3                    [-1, 1]               7
Total params: 175
Trainable params: 175
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [96]:
batch_size = 64
batches = 1945 // batch_size

data_loader = torch.utils.data.DataLoader(
    list(zip(X_train, y_train)), batch_size= 64, shuffle=True, drop_last=False
)
val_data_loder = torch.utils.data.DataLoader(
    list(zip(X_val, y_val)), batch_size=64, shuffle=True, drop_last=False
)
epochs = 300
count = 0
train_losses = []
val_losses = []
best_val_loss = float('inf')
for epoch in range(epochs):

    running_loss = 0
    running_loss_val = 0
    model.train()
    for i in range(0, len(X_train_torch), batch_size):
        X_batch = X_train_torch[i:i + batch_size]
        y_batch = y_train_torch[i:i + batch_size]
        optimizer.zero_grad()

        loss_evaluated = criterion(model(X_batch), y_batch)
        loss_evaluated.backward()

        optimizer.step()

        #running_loss += loss_evaluated
        running_loss += loss_evaluated.item() * len(X_batch)
    avg_loss = running_loss / len(X_train_torch)
    print(f"loss in train set over epoch {epoch}", avg_loss)
    model.eval()
    with torch.no_grad():
        # y_pred_tr = model(X_train)
        # predictions_tr = torch.argmax(y_pred_tr, axis=1).detach().numpy()
        # correct_tr = [1 if p == p_true else 0 for p, p_true in zip(predictions_tr, y_train)]
        # accuracy_tr = sum(correct_tr) / len(correct_tr)
        # print("training_Accuracy : ", accuracy_tr)
        val_pred = model(X_val_torch)
        val_loss = criterion(val_pred, y_val_torch)
    print(f"loss in val set over epoch {epoch}", val_loss.item())
        # y_pred_val = model(X_val)
        # predictions_val = torch.argmax(y_pred_val, axis=1).detach().numpy()
        # correct_val = [1 if p == p_true else 0 for p, p_true in zip(predictions_val, y_val)]
        # accuracy_val = sum(correct_val) / len(correct_val)
        # print("Validation_Accuracy : ", accuracy_val)
    #print(f"Accuracy: {accuracy * 100}%")
    train_losses.append(avg_loss)
    val_losses.append(val_loss.item())
    scheduler.step()
    print(f"Learning rate after epoch {epoch+1}: {scheduler.get_last_lr()[0]}")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model.state_dict()



# model.eval()
# with torch.no_grad():
#     y_pred = model(X_test)
#     predictions = torch.argmax(y_pred, axis=1).detach().numpy()
# correct = [1 if p == p_true else 0 for p, p_true in zip(predictions, Y_test)]
# accuracy = sum(correct) / len(correct)
# print(f"Accuracy: {accuracy * 100}%")

loss in train set over epoch 0 0.9911756395374288
loss in val set over epoch 0 0.9212707281112671
Learning rate after epoch 1: 0.001
loss in train set over epoch 1 0.9517636005553611
loss in val set over epoch 1 0.8908469676971436
Learning rate after epoch 2: 0.001
loss in train set over epoch 2 0.9236346551262627
loss in val set over epoch 2 0.8662493228912354
Learning rate after epoch 3: 0.001
loss in train set over epoch 3 0.8974678071421646
loss in val set over epoch 3 0.8403849005699158
Learning rate after epoch 4: 0.001
loss in train set over epoch 4 0.867504536952335
loss in val set over epoch 4 0.8080334663391113
Learning rate after epoch 5: 0.001
loss in train set over epoch 5 0.8305250045879334
loss in val set over epoch 5 0.7664425373077393
Learning rate after epoch 6: 0.001
loss in train set over epoch 6 0.7859810106552045
loss in val set over epoch 6 0.7171018123626709
Learning rate after epoch 7: 0.001
loss in train set over epoch 7 0.7367355364453824
loss in val set over

In [97]:
model.load_state_dict(best_model)
model.eval()
with torch.no_grad():
    y_test_pred_scaled = model(X_test_torch)
    y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled.cpu().numpy())

In [98]:
y_test_pred[:10], y_test_pred_scaled[:10], y_val_torch[:10]

(array([[13.170453 ],
        [ 7.861731 ],
        [26.640753 ],
        [ 3.3612034],
        [ 9.958742 ],
        [12.81683  ],
        [19.45771  ],
        [27.440052 ],
        [33.34973  ],
        [14.5256195]], dtype=float32),
 tensor([[-0.0330],
         [-0.5541],
         [ 1.2894],
         [-0.9960],
         [-0.3483],
         [-0.0677],
         [ 0.5843],
         [ 1.3679],
         [ 1.9480],
         [ 0.1001]], device='cuda:0'),
 tensor([[-1.0118],
         [-0.4473],
         [ 0.0681],
         [ 0.6375],
         [ 2.0610],
         [ 1.4719],
         [-1.0167],
         [-0.9332],
         [-0.4424],
         [ 0.8338]], device='cuda:0'))

In [99]:
mlp_submit = pd.DataFrame({
    'Id': df_test.index,
    'TARGET': y_test_pred.flatten()
})
mlp_submit.to_csv('quantum.csv', index=False)